### Install Library

In [1]:
!pip install deepface
!pip install matplotlib
!pip install opencv-python
!pip install fpdf

     |████████████████████████████████| 65 kB 2.4 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 
     |████████████████████████████████| 60.5 MB 1.2 MB/s 
     |████████████████████████████████| 2.3 MB 65.8 MB/s 
     |████████████████████████████████| 462 kB 71.7 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=5387d9828504e5db16fcebb32efd491518608b57d2206cb2bd924e46d871d328
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have im

### Import Library

In [2]:
from deepface import DeepFace
from fpdf import FPDF
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


### Salin data dari github ke local drive

In [3]:
!git clone https://github.com/33220311/DeepFace

Cloning into 'DeepFace'...
remote: Enumerating objects: 170, done.
remote: Total 170 (delta 0), reused 0 (delta 0), pack-reused 170
Receiving objects: 100% (170/170), 65.68 MiB | 59.89 MiB/s, done.
Resolving deltas: 100% (8/8), done.


### Pilihan model, metrics, dan detector yang akan digunakan

In [4]:
detectors = ["opencv", "ssd", "mtcnn", "dlib", "retinaface"]
metrics = ["cosine", "euclidean", "euclidean_l2"]
models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "Dlib", "ArcFace"]

### Path dataset testing dan kumpulan foto yang akan menjadi ground truth

In [5]:
source = '/content/DeepFace/Database'
folder ='/content/DeepFace/Test Dataset'

### Fungsi untuk menampilkan seluruh dataset Anggota kelas yang diambil dari folder Anggota kelas

In [6]:
def load_images_from_folder(folder):
    images = []
    labels=[]    
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        #if img is not None:
        images.append(img)
        labels.append(Path(filename).stem)
    return images,labels
x,lbl=load_images_from_folder(folder)
countData=len(x)

### Fungsi untuk mencetak data ke file PDF

*   setiap baris menampilkan nama file pada data test dan nama file yang dianggap mirip dengan data test serta hasil prediksinya (benar atau salah)
*   baris terakhir menampilkan akurasi untuk model, detector, dan metric yang digunakan



In [7]:
def createFile(cetak, filename, accuracy):
  pdf = FPDF()
  pdf.add_page()
  pdf.set_font("Arial", size = 15)
  str1 = "      "
  for i in range(len(cetak)):
    teks = str1.join(cetak[i])
    pdf.cell(200, 10, txt = teks, ln = 1, align = 'L')
  
  pdf.cell(200, 10, txt = "accuracy = "+ accuracy, ln = 1, align = 'L')
  pdf.output(filename)   

### Fungsi untuk menjalankan model tertentu dengan detector dan metric tertentu


*   Pengenalan dihitung benar jika wajah yang ada pada test data dapat dikenali sesuai dengan referensi utama dan wajah yang tidak ada di referensi utama dikategorikan sebagai ‘unknown’.
*   Nilai akurasi dihitung dengan membagi jumlah benar dengan total foto pada test data



In [8]:
def runModel(folder, source, model, metric, detector):
  cetak = []
  true = 0
  false = 0

  for filename in os.listdir(folder):
    name = filename.split("-")

    file = os.path.join(folder,filename)
    df = DeepFace.find(img_path=file, db_path = source, model_name = model, distance_metric = metric, detector_backend = detector, enforce_detection=False)
  
    if df.empty:
      result = "unknown"
    else:
      result = df['identity'][0]
      result = result.split("/")
      result = result[len(result)-1]
      result = result.split(" ")
      result = result[0]

    if result.lower() == name[0].lower():
      true = true + 1
      prediction ="benar"
    else:
      false = false + 1
      prediction = "salah"

    cetak.append([filename,result,prediction])
  
  accuracy = str((true / (true+false)) * 100)
  return cetak, accuracy

### Eksplorasi


*   Mencoba berbagai kombinasi model pengenalan wajah dan deteksi wajah untuk mencari model dengan kinerja terbaik.
*   Hasil prediksi disimpan dalam list kemudian dituliskan ke dalam file PDF
* Kinerja diukur menggunakan metric akurasi
* Kinerja masing-masing model disimpan dalam list


In [ ]:
hasil = []
for i in range(len(models)):
  model = models[i]
  for j in range(len(detectors)):
    detector = detectors[j]
    for k in range(len(metrics)):
      metric = metrics[k]

      cetak,acc = runModel(folder, source, model, metric, detector)
      hasil.append([model, metric, detector,acc])
      outputfile = model+"-"+detector+"-"+metric+".pdf"
      #outputfile = model+"-"+detector+"-"+metric+"-"+acc+".pdf"
      createFile(cetak, outputfile, acc)

vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:06<00:00, 87.2MB/s]


Representations stored in  /content/DeepFace/Database / representations_vgg_face.pkl  file. Please delete this file when you add new identities in your database.
find function lasts  13.459318161010742  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.28332018852233887  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.27367711067199707  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.2389669418334961  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.21927595138549805  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.27931761741638184  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.274914026260376  seconds
There are  17  representations found in  representations_vgg_face.pkl
fi

Downloading...
From: https://github.com/opencv/opencv/raw/3.4.0/samples/dnn/face_detector/deploy.prototxt
To: /root/.deepface/weights/deploy.prototxt
28.1kB [00:00, 26.1MB/s]                   


res10_300x300_ssd_iter_140000.caffemodel will be downloaded...


Downloading...
From: https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel
To: /root/.deepface/weights/res10_300x300_ssd_iter_140000.caffemodel
100%|██████████| 10.7M/10.7M [00:00<00:00, 120MB/s]


find function lasts  0.9392387866973877  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.25354504585266113  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.26336216926574707  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.23833966255187988  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.22073721885681152  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.2311694622039795  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.2580690383911133  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.8988897800445557  seconds
There are  17  representations found in  representations_vgg_face.pkl
find function lasts  0.2269299030303955  sec

In [ ]:
hasil